Teams presents files in two formats: docx and vtt.

- **docx** files include profile pics & names, time (from start, mm:ss), utterances and meta information (X joined, Y left, recording started, etc.)

- **vtt** (subtitle format) files include time (from start, HH:mm:ss.SSS) and utterances.

As of now, I work with docx files only as we need the speakers' names.

## Prepartion: installing packages & creating functions

Uncomment this cell for better view in google colab.

In [ ]:
# from IPython.display import HTML, display

# def set_css():
#   display(HTML('''
#   <style>
#     pre {
#         white-space: pre-wrap;
#     }
#   </style>
#   '''))
# get_ipython().events.register('pre_run_cell', set_css)

In [1]:
! pip install -q openai docx simplify_docx tiktoken gdown


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import docx
import re
import openai
import tiktoken
import gdown
from tqdm.notebook import tqdm
from simplify_docx import simplify
from tqdm.notebook import tqdm

In [ ]:
names_eng_to_ru = {
    "Fedor Ignatov": "Федор Игнатов",
    "Diliara Zharikova": "Диляра Жарикова",
    "Maxim Talimanchuk": "Максим Талиманчук",
    "Artem Klementev": "Артем Клементьев",
    "Nika Smilga": "Ника Смилга",
    "Timofey Syromyatnikoff": "Тимофей Сыромятников",
    "Al Popov": "Александр Попов",
    "Daniel Kornev": "Данила Корнев",
    "Irina Nikitenko": "Ирина Никитенко"
}

In [ ]:
def get_text(json_part):
    str_to_return = json_part["VALUE"][-1]["VALUE"]
    return str_to_return

In [ ]:
def structure_data(meeting_transcipt):
    meeting_transcipt_dict = {"datetime": '', "people_present": [], "utterances": []}
    for n, item in enumerate(meeting_transcipt):
        if n == 1:
            meeting_transcipt_dict["datetime"] = item
        elif "joined the meeting" in item:
            person_name = item.replace(" joined the meeting", "").strip()
            meeting_transcipt_dict["people_present"].append(names_eng_to_ru[person_name])
        elif "   " in item:
             item = [x for x in re.split(r'   |\r', item) if x]
             person = names_eng_to_ru[item[0]]
             time = item[1]
             utts = ' '.join(item[2:])
             one_utt = {"time_start": time, "person": person, "sentences": utts}
             meeting_transcipt_dict["utterances"].append(one_utt)
    return meeting_transcipt_dict

In [ ]:
def make_transcript(dict_data):
    if "utterances" in dict_data:
        utterances = dict_data["utterances"]
    else:
        utterances = dict_data
    transcript_list = [f'{utt["person"]}: {utt["sentences"]}' for utt in utterances]
    return transcript_list

In [ ]:
def check_token_number(text):
    enc = tiktoken.encoding_for_model("gpt-4")
    return len(enc.encode(text))

In [ ]:
def decide_where_to_break(transcript_list, limit=3000):
    len_tokens = 0
    break_points = []
    for n, utt in enumerate(transcript_list):
        len_tokens += check_token_number(utt)
        if len_tokens > limit:
            len_tokens = check_token_number(utt)
            break_points.append(n-1)
    return break_points

In [ ]:
def split_transcript_into_chunks(transcript, break_points):
    transcript_chunks = []
    start_point = 0
    for break_point in break_points:
        chunk = transcript[start_point:break_point]
        transcript_chunks.append(chunk)
        start_point = break_point
    return transcript_chunks

NB: gpt-4 also has a larger context window with a maximum size of **8,192** tokens compared to **4,096** tokens for gpt-3.5-turbo. However, gpt-3.5-turbo returns outputs with lower latency and costs much less per token.

In [ ]:
default_system_message = "Ты помощник офисного работника. Ты пересказываешь видео-встречи."
default_prompt = f"""Встреча:
Определи задачи, поставленные в ходе встречи для каждого сотрудника. Определи уже выполненные задачи каждого сотрудника.
Пиши только список поставленных и список выполненных задач без дополнительной информации."""


def send_prompt_to_openai(prompt=default_prompt, system_message=default_system_message, openai_model="gpt-3.5-turbo"):
    response = openai.ChatCompletion.create(
    model=openai_model,
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt},
    ]
    )
    response_text = response["choices"][-1]["message"]["content"]
    return response_text

In [ ]:
def process_meeting_chunks(transcript_chunks, prompt_first, prompt_others):
    all_responses = []
    for n, chunk in tqdm(enumerate(transcript_chunks)):
        chunk_text = '\n'.join(chunk)
        if n == 0:
            prompt = prompt_first.replace('{chunk_text}', chunk_text)
        else:
            prompt = prompt_others.replace('{response}', response).replace('{chunk_text}', chunk_text)
        response = send_prompt_to_openai(prompt=prompt)
        all_responses.append(response)
    return all_responses

In [60]:
def process_meeting_teams(filepath, prompt_first, prompt_others):
    my_doc = docx.Document(filepath)
    my_doc_as_json = simplify(my_doc)
    list_data = [get_text(json_part) for json_part in my_doc_as_json['VALUE'][0]['VALUE']]
    dict_data = structure_data(list_data)
    transcript = make_transcript(dict_data)
    break_points = decide_where_to_break(transcript)
    transcript_chunks = split_transcript_into_chunks(transcript, break_points)
    result = process_meeting_chunks(transcript_chunks, prompt_first, prompt_others)
    return result[-1]

## Prompts

In [ ]:
prompt_first_summary = """Часть встречи:
{chunk_text}
Перескажи встречу основываясь на записи. Упомяни все важные моменты, которые обсуждались в ходе встречи.
Приводи только пересказ, без дополнительных деталей."""

prompt_others_summary = """Пересказ встречи:
{response}
Исправь и дополни пересказ встречи, основываясь на этой записи:
{chunk_text}
Приводи только пересказ, без дополнительных деталей."""

prompt_first_current_tasks = """Часть встречи:
{chunk_text}
Напиши список текущих задач для каждого участника встречи.
Приводи только один список текущих задач для каждого участника, без дополнительных деталей."""

prompt_others_current_tasks = """{response}
Исправь и дополни список текущих задач  для каждого участника, основываясь на этой записи:
{chunk_text}
Приводи только один список текущих задач для каждого участника, без дополнительных деталей."""

prompt_first_completed_tasks = """Часть встречи:
{chunk_text}
Напиши список завершенных задач для каждого участника встречи.
Приводи только один список завершенных задач для каждого участника, без дополнительных деталей."""

prompt_others_completed_tasks = """{response}
Исправь и дополни список завершенных задач для каждого участника, основываясь на этой записи:
{chunk_text}
Приводи только один список завершенных задач для каждого участника, без дополнительных деталей."""

prompt_first = """YOUR PROMPT HERE"""

prompt_others = """YOUR PROMPT HERE"""

## Getting summaries & tasks

In [3]:
url = 'https://drive.google.com/uc?id=1gBQa5Ie3yjb49yWdaOIX5pCEE7pfVIBd'
filepath = './data/daily_sync_ru.docx'
gdown.download(url, filepath, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1gBQa5Ie3yjb49yWdaOIX5pCEE7pfVIBd
To: /Users/veronicasmilga/Desktop/Sentius/dream/prototypes/data/daily_sync_ru.docx
100%|██████████| 97.2k/97.2k [00:00<00:00, 2.02MB/s]


'./data/daily_sync_ru.docx'

In [ ]:
openai.api_key = "INSERT YOUR KEY HERE"

In [ ]:
summary = process_meeting_teams(filepath, prompt_first_summary, prompt_others_summary)
print(summary)

0it [00:00, ?it/s]

На встрече обсуждали вопросы, связанные с разработкой системы для сбора информации о компании и организации работы менеджеров. Была предложена идея создать отдельную папку для хранения файлов и обсуждены проблемы с гугл-аккаунтом и докер-контейнерами. Также был обсужден план развития системы, включающий внедрение спринтов и митингов. Решено было создать канал "research" в разделе инжиниринга и дизайна для обсуждения и публикации релевантной информации. Также обсуждалась необходимость подключения к системе микрософт аккаунта для авторизации сотрудников. Некоторые участники предложили сначала пробный вариант, а затем перейти к правильному пути реализации.


In [ ]:
current_tasks = process_meeting_teams(filepath, prompt_first_current_tasks, prompt_others_current_tasks)
print(current_tasks)

0it [00:00, ?it/s]

- Данил Корнев: создание гугл аккаунта для свинтуса, создание гугл клайн секрета для проекта, создание гугл open kids, исправление переменных чтобы сервисы не падали
- Диляра Жарикова: работа над инструкцией по лекторам, проверка и настройка пыль квест короче, локальная проверка и исправление локального выбора критерия в леон бет спорт
- Максим: исправление переменных чтобы сервисы не падали
- Ирина Никитенко: узнать о костелях
- Федор: узнать, что такое вики
- Артём Клементьев: разработка вкладки со скитлс ректором и response ректором, ожидание данных от Макса для редактирования скилл с электрофорез spn селектора


In [ ]:
completed_tasks = process_meeting_teams(filepath, prompt_first_completed_tasks, prompt_others_completed_tasks)
print(completed_tasks)

0it [00:00, ?it/s]

Диляра Жарикова: Создание колонки ревью
Александр Попов: Постановка задачи
Данила Корнев: Разработка интерфейса
Ирина Никитенко: -
Ника Смилга: -
